In [ ]:
#General imports
import math
import numpy as np
import time
from operator import itemgetter
import scipy.cluster.hierarchy as hcluster #for clustering ball positions
from collections import Counter
from bisect import bisect_left #For organizing timestamps

#Imports from ros and opencv
import rospy
import cv2
from sensor_msgs.msg import Image
from aruco_msgs.msg import MarkerArray
from cv_bridge import CvBridge, CvBridgeError
from geometry_msgs.msg import Twist
from nav_msgs.msg import Odometry
from std_msgs.msg import String

#Imports for plotting
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
%matplotlib inline

#Initializing global variables and classes
bridge = CvBridge()
rospy.init_node('turtlebot_singleplayer', anonymous=True)
rospy.sleep(0.5)
lower_blue = (0,0,120)
upper_blue = (100,100,255)
lower_red = (200,0,0)
upper_red = (255,100,100)

###################
#Callback functions
###################
def callback_color(msg):
    global color_timestamp
    global color_img
    color_timestamp = msg.header.stamp
    color_img = bridge.imgmsg_to_cv2(msg, "rgb8")

def callback_depth(msg):
    global depth_img
    global depth_timestamp
    depth_timestamp = msg.header.stamp
    depth_img = bridge.imgmsg_to_cv2(msg, "32FC1")

def callback_odom(data):
    global x
    global y
    global theta
    global odom_timestamp 
    position = data.pose.pose.position
    orientation = data.pose.pose.orientation
    odom_timestamp = data.header.stamp
    x = round(position.x, 3)
    y = round(position.y, 3)
    theta = round(2*math.atan2(orientation.z,orientation.w), 3)
    if theta<0:
        theta = 2*math.pi + theta

def callback_array(msg):
    global marker_array
    global marker_array_timestamp
    marker_array_timestamp = msg.header.stamp
    marker_array = msg.markers
    #print "Marker array:",marker_array_timestamp

def callback_state(msg):
    global dist2_balls 
    global dist2_gate
    global theta2_dif
    global state2
    state_dict = msg
    if(state_dict == 'measuring'):
        state2 = 'measuring'
    else:
        state_dict = eval(state_dict)
        dist2_balls = state_dict['dist_balls']
        dist2_gate = state_dict['dist_gate']
        theta_dif = state_dict['theta_dif']
##########################
#Functions used in general
##########################

#turn to certain theta
def turnToTheta(theta_tar):
    
    #Checking theta value
    if theta_tar < 0:
        theta_tar += math.pi *2
    if theta_tar > math.pi*2:
        theta_tar -= math.pi *2
        
    twist = Twist()
    twist.linear.x = 0.0
    ang_diff = theta_tar - theta
    s = np.sign(ang_diff)
    if abs(ang_diff) > math.pi:
        s *= -1.0
        ang_diff = 2*math.pi - abs(ang_diff)
    ang_vel = s*ang_diff*0.6
    if abs(ang_vel) < 0.2:
        ang_vel = np.sign(ang_vel)*0.2
    twist.angular.z = ang_vel
    r = rospy.Rate(10)
    while(abs(theta_tar-theta) > 0.1):
        publisher.publish(twist)
        r.sleep()
        ang_diff = abs(theta_tar - theta)
        if ang_diff > math.pi:
            ang_diff = 2*math.pi - ang_diff
        ang_vel = s*ang_diff*0.6
        if abs(ang_vel) < 0.2:
            ang_vel = np.sign(ang_vel)*0.2
        twist.angular.z = ang_vel
        #print theta

def moveForward(distance):
    #calibrate
    distance = (distance - 0.14)/0.85
    t = int(round(distance*25))
    twist = Twist()
    twist.linear.x = 0.4
    twist.angular.z = 0.0
    r = rospy.Rate(10)
    for i in range(t):
        publisher.publish(twist)
        r.sleep()

#Given color and depth image calculate the depth point, and middle points of ball
def get_depth(colorim,depthim):
    mask_blue = cv2.inRange(colorim, lower_blue, upper_blue)
    mask_red = cv2.inRange(colorim, lower_red, upper_red)
    kernel = np.ones((5,5),np.uint8)
    erosion_blue = cv2.erode(mask_blue,kernel,iterations = 1)
    dilation_blue = cv2.dilate(erosion_blue,kernel,iterations = 3)
    erosion_red = cv2.erode(mask_red,kernel,iterations = 1)
    dilation_red = cv2.dilate(erosion_red,kernel,iterations = 3)
    mask = cv2.bitwise_or(dilation_blue, dilation_red)
    im, cnt, hrc = cv2.findContours(mask,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    cnt.sort(key=cv2.contourArea, reverse=True)
    if cnt:
        x, y, w, h = cv2.boundingRect( cnt[0] )
        if (w*h > 2000):
            x_ball = x+w/2
            y_ball = y+h/2
            dist_ball = np.nanmedian(depthim[y:y+h,x:x+w])
        else:
            x_ball = 0
            y_ball = 0
            dist_ball = None
    else:
        x_ball = 0
        y_ball = 0
        dist_ball = None
    return x_ball,y_ball,dist_ball

#For organizing measurements by timestamp
def takeClosest(myList, myNumber):
    """
    Assumes myList is sorted. Returns closest value to myNumber.

    If two numbers are equally close, return the smallest number.
    """
    pos = bisect_left(myList, myNumber)
    if pos == 0:
        return myList[0]
    if pos == len(myList):
        return myList[-1]
    before = myList[pos - 1]
    after = myList[pos]
    if after - myNumber < myNumber - before:
       return after
    else:
       return before


#Subscribers and Publishers
subscriber_color = rospy.Subscriber("/bestRobot/camera/rgb/image_rect_color", Image, callback_color)
subscriber_depth = rospy.Subscriber("/bestRobot/camera/depth/image_rect", Image, callback_depth)
subscriber = rospy.Subscriber("/bestRobot/odom", Odometry, callback_odom)
subscriber_array = rospy.Subscriber("/bestRobot/aruco_marker_publisher/markers", MarkerArray, callback_array)
publisher = rospy.Publisher('/bestRobot/cmd_vel_mux/input/teleop', Twist, queue_size=10)

#Subscriber publisher to connect to the other
publisher_state = rospy.Publisher('/bestRobot/state', String, queue_size=1)
subscribe_state = rospy.Publisher('/state', String, callback_state)

In [ ]:
#################
#Finding the gate
#################
publisher_state.publish("measuring")

gate_dist, gate_stamp, gate_theta, gate_theta_raw, gate_theta_stamp = [],[],[],[],[] # Whether it was found, and its theta coordinate, and x 
twist = Twist()
#We need this if we want to turn full circle
theta_initial = theta
theta_opposite = math.atan2(-math.sin(theta_initial),-math.cos(theta_initial))
if theta_opposite < 0:
    theta_opposite += 2* math.pi
half_circle = False

#For data storage
marker_array = None
m = None
#Turn around and save data
while (abs(theta-theta_initial)>0.1 or (half_circle != True)):
    if(abs(theta-theta_opposite)<0.1):
        half_circle = True
    twist.angular.z = 0.2
    
        
    r = rospy.Rate(100)
    for i in range(10):
        publisher.publish(twist)
        r.sleep()
    
    gate_theta_raw.append(theta)
    gate_theta_stamp.append(odom_timestamp)
    try:
        m = marker_array[0]
        gate_dist.append(math.sqrt(m.pose.pose.position.x**2+m.pose.pose.position.z**2))
        gate_stamp.append(marker_array_timestamp)
        marker_array = None
    except TypeError:
        pass
    
#Organizing timestamps
for stamp in gate_stamp:
    index = gate_theta_stamp.index(takeClosest(gate_theta_stamp, stamp))
    gate_theta.append(gate_theta_raw[index])
    
#Theta cleanup in (x,y) space
x_ = np.cos(gate_theta)
y_ = np.sin(gate_theta)
theta_gate = math.atan2(np.mean(y_),np.mean(x_))
if theta_gate < 0:
    theta_gate += 2*math.pi
#Distance cleanup
sd = np.std(gate_dist)
mean = np.mean(gate_dist)
final_list = [x for x in gate_dist if (x > mean - sd)]
final_list = [x for x in final_list if (x < mean +  sd)]
dist_gate = np.mean(final_list)

#Calibrating the distance of the gate
dist_gate = dist_gate * 1.13 + 0.02

#################
#Finding the ball
#################
twist = Twist()
#We need this if we want to turn full circle
theta_initial = theta
theta_opposite = math.atan2(-math.sin(theta_initial),-math.cos(theta_initial))
if theta_opposite < 0:
    theta_opposite += 2* math.pi
half_circle = False

ball_pos = []
#Turn around
while (abs(theta-theta_initial)>0.1 or (half_circle != True)):
    if(abs(theta-theta_opposite)<0.1):
        half_circle = True
        
    twist.angular.z = 0.2
    r = rospy.Rate(100)
    for i in range(5):
        publisher.publish(twist)
        r.sleep()
    theta_cur = theta
    colorim = color_img.copy()
    depthim = depth_img.copy()
    x_ball, y_ball, dist_ball = get_depth(colorim,depthim)
    if dist_ball:
        ball_pos.append([x_ball, y_ball, theta_cur, dist_ball])
    
#Cleanup theta
ball_np = np.array(ball_pos)
#Remove nan values
ball_np = ball_np[~np.isnan(ball_np).any(axis=1)]
#Go to sin and cos space from angle
ball_temp = np.empty([ball_np.shape[0],4])
ball_temp[:,0] = np.sin(ball_np[:,2])
ball_temp[:,1] = np.cos(ball_np[:,2])
ball_temp[:,2] = ball_np[:,3]
ball_temp[:,3] = ball_np[:,2]
#Do the clustering
thresh = 0.2
clusters = hcluster.fclusterdata(ball_temp, thresh, criterion="distance")
#Separate clusters
b = Counter(clusters)
label = b.most_common(1)[0][0]
theta_cleaned = []
for ind,cl in enumerate(clusters):
    if cl == label:
        theta_cleaned.append([ball_temp[ind,0],ball_temp[ind,1],ball_temp[ind,2],ball_temp[ind,3]])
#Caluclate average theta
theta_ball = math.atan2(np.mean(np.array(theta_cleaned)[:,0]),np.mean(np.array(theta_cleaned)[:,1]))
if theta_ball < 0:
    theta_ball += 2*math.pi
dist_balls = np.mean(np.array(theta_cleaned)[:,2])

# #Calibrating the distance
dist_balls = dist_balls *1.01 + 0.07

In [ ]:
def calculateOtherPosition(theta_ball,dist_balls,theta_gate,dist_gate,
                          dist2_balls,dist2_gate,theta2_diff):
    #Theta dif: difference between the theta of the (theta_ball-theta_gate)
    
    #Go to Cartesian coordinates
    xb = math.cos(theta_ball) * dist_balls
    yb = math.sin(theta_ball) * dist_balls
    xg = math.cos(theta_gate) * dist_gate
    yg = math.sin(theta_gate) * dist_gate
    
    #Caluclate the two points
    m = (xb-xg)/(yg-yb)
    k = (dist2_balls**2 - dist2_gate**2 + xg**2 - xb**2 + yg**2 - yb**2)/(2*(yg-yb))
    
    a = 1 + m**2
    b = 2*(m*k - m*yg -xg)
    c = (xg**2 + yg**2 + k**2 - 2*k*yg - dist2_gate**2)
    
    x1=((-b)+math.sqrt(b**2-4*a*c))/(2*a);#x=(-b+√(b^2-4ac))/2a
    y1=m*x1+k; #y=mx+k
    x2=((-b)-math.sqrt(b**2-4*a*c))/(2*a);#x=(-b-√(b^2-4ac))/2a
    y2=m*x2+k; #y=mx+k
    
    #Decide which one is the correct one
    if math.atan2(y1-yg,x1-xg) < math.atan2(y1-yb,x1-xb):
        if theta2_diff > 0:
            theta3 = math.atan2(y1,x1)
            dist3 = math.sqrt(x1**2 + y1**2)
        else:
            theta3 = math.atan2(y2,x2)
            dist3 = math.sqrt(x2**2 + y2**2)
    else:
        if theta2_diff > 0:
            theta3 = math.atan2(y2,x2)
            dist3 = math.sqrt(x2**2 + y2**2)
        else:
            theta3 = math.atan2(y1,x1)
            dist3 = math.sqrt(x1**2 + y1**2)
    return dist3, theta3

In [ ]:
#Plotting the theta and distance values for the ball
plt.scatter(ball_np[:,2],ball_np[:,3],c=clusters,s=500,marker='o')
plt.title("Ball Position Clusters")
plt.xlabel("Theta Angle [rad]")
plt.ylabel("Ball Distance [m]")
plt.show()

#Publish the data which was acquired
theta_dif = gate-ball
if(theta_dif<0):
    theta_dif += math.pi*2
state_dict = {'dist_balls':dist_balls,'dist_gate':dist_gate,'theta_dif':theta_dif,'recieved':False}
while(state_dict['recieved'] == False):
    try:
        if(dist2_balls):
            state_dict['recieved'] = True
    except NameError:
        pass
    publisher_state.publish(str(state_dict))
    rospy.sleep(0.5)
print calculateOtherPosition(theta_ball,dist_balls,theta_gate,dist_gate,
                          dist2_balls,dist2_gate,theta2_dif)

In [ ]:
##################
#Diagnostic figure
##################
plt.figure(figsize=(10, 10))
ax = plt.axes(polar=True)
max_val = math.ceil(max(dist_ball, dist_gate,max(np.nan_to_num(ball_np[:,3])),max(gate_dist) ))
handle = []

#Plot robot
ax.plot(0, 0,marker='o',markersize=60)
ax.text( s =r'TURTLEBOT',x=0,y=0, horizontalalignment='center',
        verticalalignment='center')
handle.append(mlines.Line2D([], [], color='blue', marker='o',
                          markersize=15, label='Robot Position'))


#PlotGate
try:
    ax.plot(theta_gate,dist_gate,marker='o',markersize=70)
    ax.text( s =r'GATE',x=theta_gate,y=dist_gate, horizontalalignment='center',
            verticalalignment='center')
    handle.append(mlines.Line2D([], [], color='green', marker='o',
                          markersize=15, label='Gate Position'))
except:
    pass

#PlotBall
try:
    ax.plot(theta_ball,dist_balls,marker='o',markersize=45)
    ax.text( s =r'BALL',x=theta_ball, y=dist_balls, horizontalalignment='center',
            verticalalignment='center')
    handle.append(mlines.Line2D([], [], color='red', marker='o',
                          markersize=15, label='Ball Position'))
except:
    pass

#Data points of ball
try:
    y_ball = np.nan_to_num(ball_np[:,3])+np.isnan(ball_np[:,3])*(max_val*0.8)
    ax.plot(ball_np[:,2],y_ball,marker='D',markersize=5,c='purple')
    handle.append(mlines.Line2D([], [], color='purple', marker='D',
                          markersize=15, label='Ball Measurements'))
    ax.plot(np.array(theta_cleaned)[:,3],np.array(theta_cleaned)[:,2],marker='s',markersize=5,c='orange')
    handle.append(mlines.Line2D([], [], color='orange', marker='s',
                          markersize=15, label='Ball Filtered'))
except:
    pass
                    
#Data points of gate
try:
    ax.plot(gate_theta,gate_dist,marker='o',markersize=5,c='gray')
    handle.append(mlines.Line2D([], [], color='gray', marker='o',
                          markersize=15, label='Gate Measurements'))
except:
    pass

ax.set_rmax(max_val)
ax.set_rlabel_position(0)  # get radial labels away from plotted line
ax.grid(True)

ax.set_title("Objects around the TURTLEBOT", va='bottom')
xL = [0,0.25,0.5,0.75,1,1.25,1.5,1.75]
xL = [round(math.pi*x,2) for x in xL]
ax.set_xticklabels(xL, fontdict=None, minor=False)
plt.legend(handles=handle)

plt.show()

In [ ]:
#calculate angle and distance to go to the position to kick the ball
dist_BT = 0.5
ang_GRB = abs(theta_gate - theta_ball)
if ang_GRB > math.pi:
    ang_GRB = 2*math.pi - ang_GRB
dist_RB = dist_balls
dist_RG = dist_gate
dist_BG = math.sqrt(pow(dist_RG,2) + pow(dist_RB,2) - 2*dist_RG*dist_RB*math.cos(ang_GRB))
ang_GBR = math.acos((pow(dist_BG,2) + pow(dist_RB,2) - pow(dist_RG,2)) / (2*dist_BG*dist_RB))
ang_RBT = math.pi - ang_GBR
dist_RT = math.sqrt(pow(dist_RB,2) + pow(dist_BT,2) - 2*dist_RB*dist_BT*math.cos(ang_RBT))
ang_BRT = math.acos((pow(dist_RB,2) + pow(dist_RT,2) - pow(dist_BT,2)) / (2*dist_RB*dist_RT))
ang_BTR = math.acos((pow(dist_BT,2) + pow(dist_RT,2) - pow(dist_RB,2)) / (2*dist_BT*dist_RT))
ang_back = math.pi - ang_BTR
if abs(theta_gate - theta_ball) > math.pi:
    if theta_ball > theta_gate:
        theta_tar = theta_ball - ang_BRT
        theta_back = theta_tar + ang_back
    else:
        theta_tar = theta_ball + ang_BRT
        theta_back = theta_tar - ang_back
else:
    if theta_ball < theta_gate:
        theta_tar = theta_ball - ang_BRT
        theta_back = theta_tar + ang_back
    else:
        theta_tar = theta_ball + ang_BRT
        theta_back = theta_tar - ang_back

In [ ]:
def kickPosition(theta_target, dist_target, theta_second, dist_second, Pass):
    dist_BT = 0.5
    ang_GRB = abs(theta_target - theta_second)
    if ang_GRB > math.pi:
        ang_GRB = 2*math.pi - ang_GRB
    dist_RB = dist_second
    dist_RG = dist_target
    dist_BG = math.sqrt(pow(dist_RG,2) + pow(dist_RB,2) - 2*dist_RG*dist_RB*math.cos(ang_GRB))
    ang_GBR = math.acos((pow(dist_BG,2) + pow(dist_RB,2) - pow(dist_RG,2)) / (2*dist_BG*dist_RB))
    ang_RBT = math.pi - ang_GBR
    dist_RT = math.sqrt(pow(dist_RB,2) + pow(dist_BT,2) - 2*dist_RB*dist_BT*math.cos(ang_RBT))
    ang_BRT = math.acos((pow(dist_RB,2) + pow(dist_RT,2) - pow(dist_BT,2)) / (2*dist_RB*dist_RT))
    ang_BTR = math.acos((pow(dist_BT,2) + pow(dist_RT,2) - pow(dist_RB,2)) / (2*dist_BT*dist_RT))
    ang_back = math.pi - ang_BTR
    if abs(theta_target - theta_second) > math.pi: # 0-2pi condition
        if Pass !=(theta_second > theta_target):
            theta_move = theta_second - ang_BRT
            theta_back = theta_move + ang_back
        else:
            theta_move = theta_second + ang_BRT
            theta_back = theta_move - ang_back
    else:
        if Pass != (theta_second < theta_target):
            theta_move = theta_second - ang_BRT
            theta_back = theta_move + ang_back
        else:                          
            theta_move = theta_second + ang_BRT
            theta_back = theta_move - ang_back
    return theta_move, dist_RT, theta_back

In [ ]:
#Pass the ball to teammate
theta_middle, dist_middle, theta_nouse = kickPosition(theta_gate, dist_gate, theta2, dist2, True)
theta_tar, dist_tar, theta_back = kickPosition(theta_middle, dist_middle, theta_ball, dist_balls, False)
turnToTheta(theta_tar)
moveForward(dist_tar)
turnToTheta(theta_back)
#KICK THE BALL FUNCTION

#

In [ ]:
#move to the target position
turnToTheta(theta_tar)
moveForward(dist_RT)

turnToTheta(theta_back)

#Get gate distance
rospy.sleep(0.5)
m = marker_array[0]
goal_dist = math.sqrt(m.pose.pose.position.x**2+m.pose.pose.position.z**2)
goal_dist = (goal_dist-0.2) * 1.08 + 0.03 #we stop 15 cm ahead so it wont hit the gate
moveForward(goal_dist) 